In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from random import randrange

from Functions import findHutIDs

In [6]:



#s.get('https://www.alpsonline.org/reservation/calendar?hut_id=27')
#r = s.get('https://www.alpsonline.org/reservation/selectDate?date=10.07.2024')
#json_list = []
#for i, value in r.json().items():
#    json_list.append(value[0])

#df = pd.DataFrame(json_list)

In [7]:
hutid = [27, 515]
date = '10.07.2024'


headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

s = requests.Session()
s.headers.update(headers)
#loop over all huts, create a dataframe for each hut
#initialize dataframes
df_list = []

for hut in hutid:
    r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
    r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
    json_list = []
    for i, value in r.json().items():
        json_list.append(value[0])
    df = pd.DataFrame(json_list)
    df['hutid'] = hut
    df_list.append(df)


#reduce dataframes to relevant columns, add hutid and calculate occupation
df_red_list = []
for df in df_list:
    df_red = df[['hutid', 'reservationDate', 'freeRoom']].copy()
    df_red['occupation'] = (100 - df_red['freeRoom'].copy() / df['totalRoom'].copy() * 100)
    df_red['Service'] = df['bedCategoryType'].copy()
    df_red_list.append(df_red)

#concatenate dataframes
df_con = pd.concat(df_red_list)
print(df_con)


    hutid reservationDate  freeRoom  occupation     Service
0      27      14.07.2024        36   32.075472        ROOM
1      27      15.07.2024        26   50.943396        ROOM
2      27      16.07.2024        24   54.716981        ROOM
3      27      17.07.2024        25   52.830189        ROOM
4      27      18.07.2024        40   24.528302        ROOM
5      27      19.07.2024         7   86.792453        ROOM
6      27      20.07.2024         1   98.113208        ROOM
7      27      21.07.2024        36   32.075472        ROOM
8      27      22.07.2024        27   49.056604        ROOM
9      27      23.07.2024        21   60.377358        ROOM
10     27      24.07.2024        45   15.094340        ROOM
11     27      25.07.2024        46   13.207547        ROOM
12     27      26.07.2024        35   33.962264        ROOM
13     27      27.07.2024         0  100.000000        ROOM
0     515      14.07.2024         6   40.000000  UNSERVICED
1     515      15.07.2024        10    0

In [4]:
#Read csv file with hut information
#contains: hutid, hutname, altitude
df_huts = pd.read_excel('hutInfo.xlsx')
print(df_huts)

   Unnamed: 0  Hut ID                                      Hut Name  Altitude  \
0           0       1  Blüemlisalphütte SAC, SAC Sektion Blümlisalp      2840   
1           1       2             Glecksteinhütte SAC, SAC Burgdorf      2317   
2           2       3      Capanna da l'Albigna CAS, SAC Hoher Rohn      2336   
3           3       4                    Kesch-Hütte SAC, SAC Davos      2625   

         Coordinates                                   Reservation Link  
0  625.565 / 151.040  https://www.alpsonline.org/reservation/calenda...  
1  650.400 / 163.979  https://www.alpsonline.org/reservation/calenda...  
2  770.675 / 133.375  https://www.alpsonline.org/reservation/calenda...  
3  786.561 / 168.870  https://www.alpsonline.org/reservation/calenda...  


In [ ]:
def FindBeds(date, NumPeople = 1, nights = 1, SACOnly = False):
    #Input:
        #first night of the reservation
        #look for all huts, not just SAC huts
        #number of people in the reservation
        #number of nights in the reservation
    #Output:
        #dataframe with all suitable huts, their occupation and the number of beds available

    #load hut information (hutid, hutname, altitude)
    if SACOnly:
        df_huts = pd.read_excel('HutInfo.xlsx')
    else:
        df_huts = pd.read_excel('HutInfoSAC.xlsx')

    #initialize pandas dataframe
    df_freeHuts = pd.DataFrame(columns = ['Hut ID', 'Hut Name', 'Altitude', 'Free Beds', 'Capacity', 'Service', 'Reservation Link'])

    #initialize session to grab data from alpsonline
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
       'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    }

    s = requests.Session()
    s.headers.update(headers)

    hutCounter = 0 #count how many huts we've found
    
    #loop through all hutIDs
    for index, hut in enumerate(df_huts['Hut ID']):
        r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
        r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
        json_list = []
        for i, value in r.json().items():
            json_list.append(value[0])
        df = pd.DataFrame(json_list)
        df['hutid'] = hut
        df_list.append(df)
        #go through df, check if there are enough beds available
        for j in range(0,nights):
            hasfreerooms = True
            if df['freeRoom'][j] < NumPeople:
                hasfreerooms = False
                break
        if hasfreerooms:
            #add hutid, hutname, altiude, occupation, freeRoom, Service to df_freeHuts
            df_freeHuts.loc[hutCounter] = [df_huts['Hut ID'][index], df_huts['Hut Name'][index], df_huts['Altitude'][index], df['freeRoom'][0], df['reservedRoomsRatio'][0]*100, df['contingentTitle'][0], df_huts['Reservation Link'][index]]
            hutCounter += 1

In [10]:
#look up reservations

hutid = 680
date = '10.07.2024'

r = s.get(f'https://www.alpsonline.org/reservation/calendar?hut_id={hut}')
r = s.get(f'https://www.alpsonline.org/reservation/selectDate?date={date}')
json_list = []
for i, value in r.json().items():
    json_list.append(value[0])
df = pd.DataFrame(json_list)
df['hutid'] = hut
df_list.append(df)

print(df)

    hutBedCategoryId  bedCategoryId bedCategoryType  bookingEnabled  closed  \
0               1350             43      UNSERVICED            True   False   
1               1350             43      UNSERVICED            True   False   
2               1350             43      UNSERVICED            True   False   
3               1350             43      UNSERVICED            True   False   
4               1350             43      UNSERVICED            True   False   
5               1350             43      UNSERVICED            True   False   
6               1350             43      UNSERVICED            True   False   
7               1350             43      UNSERVICED            True   False   
8               1350             43      UNSERVICED            True   False   
9               1350             43      UNSERVICED            True   False   
10              1350             43      UNSERVICED            True   False   
11              1350             43      UNSERVICED 

In [16]:
df_freeHuts = pd.DataFrame(columns = ['hutid', 'hutname', 'altitude', 'occupation', 'freeRoom', 'Service'])
df_freeHuts = df_freeHuts.append({'hutid': df_huts['Hut ID'][hut], 'hutname': df_huts['Hut Name'][hut], 'altitude': df_huts['Altitude'][hut], 'occupation': df['occupation'][0], 'freeRoom': df['freeRoom'][0], 'Service': df['bedCategoryType'][0]}, ignore_index = True)


AttributeError: 'DataFrame' object has no attribute 'append'

0     Unattended operation
1     Unattended operation
2     Unattended operation
3     Unattended operation
4     Unattended operation
5     Unattended operation
6     Unattended operation
7     Unattended operation
8     Unattended operation
9     Unattended operation
10    Unattended operation
11    Unattended operation
12    Unattended operation
13    Unattended operation
Name: contingentTitle, dtype: object

In [19]:
df_huts = pd.read_excel('HutInfo.xlsx')
df_huts

,Hut ID,Hut Name,Altitude,Coordinates,Reservation Link
0,1,"Blüemlisalphütte SAC, SAC Sektion Blümlisalp",2840,625.565 / 151.040,https://www.alpsonline.org/reservation/calenda...
1,2,"Glecksteinhütte SAC, SAC Burgdorf",2317,650.400 / 163.979,https://www.alpsonline.org/reservation/calenda...
2,3,"Capanna da l'Albigna CAS, SAC Hoher Rohn",2336,770.675 / 133.375,https://www.alpsonline.org/reservation/calenda...
3,4,"Kesch-Hütte SAC, SAC Davos",2625,786.561 / 168.870,https://www.alpsonline.org/reservation/calenda...
